<a href="https://colab.research.google.com/github/patelhiren3385/NLP/blob/master/Encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

# Instantiates the device to be used as GPU/CPU based on availability
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

import random

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/My\ Drive/YOLO/Encoder_decoder

/content/drive/My Drive/YOLO/Encoder_decoder


In [ ]:
!ls

Training_eng_hindi.xml	Validation_eng_hindi.xml


In [ ]:
eng_alphabates = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad_char = '-PAD-'

eng_char2index = {pad_char: 0}

for index,char_ in enumerate(eng_alphabates):
  eng_char2index[char_] = index+1 
print(eng_char2index)

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [ ]:
hindi_alphabates = [chr(alpha) for alpha in range(2304,2432)]
hindi_char2index = {pad_char:0}

for index,char in enumerate(hindi_alphabates):
  hindi_char2index[char] = index+1
print(hindi_char2index)

{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 108, '६': 109, '७': 

In [ ]:
import re
non_eng_letters = re.compile('[^a-zA-Z ]')

def cleanEnglishVocab(line):
  line = line.replace('-',' ').replace(',',' ').upper()
  line = non_eng_letters.sub('',line)
  return line.split()

def cleanHindiVocab(line):
  line = line.replace('-',' ').replace(',',' ')
  cleaned_line = ''
  for char in line:
    if char in hindi_char2index or char == ' ':
      cleaned_line += char
  return cleaned_line.split()

In [ ]:
import torch
import re
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET 

class TransliterationDataset(Dataset):
  def __init__(self,filename):
    self.eng_words, self.hindi_words = self.readXML(filename,cleanHindiVocab)
    self.shuffle_indices = list(range(len(self.eng_words)))
    random.shuffle(self.shuffle_indices)
    self.shuffle_start_index = 0 
  
  def __len__(self):
    return len(self.eng_words)

  def __getitem__(self,idx):
    return self.eng_words[idx],self.hindi_words[idx]

  def readXML(self,filename,hindi_vocab_clean):
    transliterationCorpus = ET.parse(filename).getroot()
    lang1_words = []
    lang2_words = []

    for line in transliterationCorpus:
      wordlist1 = cleanEnglishVocab(line[0].text)
      wordlist2 = hindi_vocab_clean(line[1].text)

      if len(wordlist1) != len(wordlist2):
        print('skip: ', line[0].text , ' - ', line[1].text)
        continue
      for word in wordlist1:
        lang1_words.append(word)
      for word in wordlist2:
        lang2_words.append(word)

    return lang1_words, lang2_words

  def get_random_sample(self):
    return self.__getitem__(np.random.randint(len(self.eng_words)))

  def get_batch(self,batch_size, postprocess = True):
    eng_batch = self.get_batch_from_array(batch_size, self.eng_words)
    hindi_batch = self.get_batch_from_array(batch_size, self.hindi_words)
    self.shuffle_start_index = 0

    #Reshuffle if one epoch is complete
    if self.shuffle_start_index >= len(self.end_words):
      random.shuffle(self.shuffle_indices)
      self.shuffle_start_index = 0

    return eng_batch, hindi_batch


In [ ]:
!ls

Training_eng_hindi.xml	Validation_eng_hindi.xml


In [ ]:
train_data = TransliterationDataset('Training_eng_hindi.xml')

skip:  BARHARWA JUNCTION  -  बरहरवा
skip:  STATE BNK TR  -  स्टेट बैंक ऑफ त्रावणकोर
skip:  SOUTH ARLINGTON CHURCH OF CHRIST  -  साउथ अर्लिंग्टन
skip:  KING EDWARD VII  -  किंग एडवर्ड
skip:  DIBANG VALLEY  -  दिबंगवैली
skip:  ORDER OF VASA  -  ऑडर ऑफ़ द वासा
skip:  AZAMNAGAR ROAD  -  आज़मनगर
skip:  CAPE TOWN  -  केपटाउन
skip:  NEW ZEALAND  -  न्यूज़ीलैंड
skip:  SEA OF THE HEBRIDES  -  सी ऑफ हरब्रिड्‍स
skip:  RAMCOIND  -  राम्को इंड
skip:  KELVINGROVE ART GALLERY AND MUSEUM  -  केल्व‍िनग्रोव आर्ट एण्ड म्युज़ियम
skip:  AUSTRALIAN NATIONAL UNIVERSITY  -  ऑस्ट्रेलियननेशनल यूनिवर्सिटी
skip:  JAHAN AARA  -  जहाँआरा
skip:  NAVABHARAT FERRO ALLOYS  -  नव भारत फ़ैरो अलॉय
skip:  RAMA LINGESHWARA  -  रामालिंगेश्वर
skip:  FAKHRUN NISA  -  फखरुन्निसा
skip:  REDIFF.COM INDIA LIMITED  -  रेडिफ़ डॉट कॉम इंडिया लिमिटेड
skip:  OMKARNATH THAKUR  -  ओंकार नाथ ठाकुर
skip:  OPENTV  -  ओपन टीवी
skip:  ENVOY COMMUNICATIONS GROUP  -  एन्वॉय कम्युनिकेशंस
skip:  WAR OF THE HOLY LEAGUE  -  वार ऑफ होली लीग
skip:  VAPA

In [ ]:
for i in range(10):
  eng, hindi = train_data.get_random_sample()
  print(eng , ' - ' , hindi)

KULLURSANDAI  -  कुल्लुरसन्दई
CHASE  -  चेस
IRSHAD  -  इरशाद
PYAR  -  प्यार
SINDOOR  -  सिंदूर
FORT  -  फोर्ट
OF  -  ऑफ
CAMPBELL  -  केम्पबेल
RESERVOIR  -  रिज़रवायर
PRADYOT  -  प्रद्योत


In [ ]:
class WordEncoding:
  def __init__(self, eng_indexing, hindi_indexing, device = 'cpu'):
    self.eng_indexing = eng_indexing
    self.hindi_indexing = hindi_indexing
    self.device = device
 
  def encode_now(self,word):
    if word[0] in self.eng_indexing:
      letter2index = self.eng_indexing
      encode = torch.zeros(len(word)+1, 1 , len(letter2index)).to(self.device)
      for index,letter_word in enumerate(word):
        pos = letter2index[letter_word]
        encode[index][0][pos] = 1
      pad_pose = letter2index[pad_char]
      encode[index+1][0][pad_pose] = 1
      return encode
    else:
      letter2index = self.hindi_indexing
      encode = torch.zeros([len(word)+1,1], dtype=torch.long).to(self.device)
      for index, letter_word in enumerate(word):
        pos = letter2index[letter_word]
        encode[index][0] = pos
      encode[index+1][0] = letter2index[pad_char]
      return encode


In [ ]:
eng, hindi = train_data.get_random_sample()
Encode_ = WordEncoding(eng_char2index,hindi_char2index)
Encode_.encode_now(eng)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

# Encoder Decoder with GRU


In [ ]:
MAX_OUTPUT_CHARS = 30
class Transliteration_EncoderDecoder(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, verbose = False):
    super(Transliteration_EncoderDecoder,self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.encoder_rnn_cell = nn.GRU(input_size,hidden_size)
    self.decoder_rnn_cell = nn.GRU(output_size,hidden_size)

    self.hidden_2_output = nn.Linear(hidden_size,output_size)
    self.softmax = nn.LogSoftmax(dim = 2)

    self.verbose = verbose

  def forward(self, input, max_output_chars = MAX_OUTPUT_CHARS, device = 'cpu', ground_truth = None):
    #Encoder
    out, hidden = self.encoder_rnn_cell(input)

    if self.verbose:
      print("Encoder Input Size: ", input.shape)
      print("Encoder Hidden Size: ", hidden.shape)
      print("Encoder Output Size: ", out.shape)

    #Decoder
    decoder_state = hidden 
    decoder_input = torch.zeros(1,1,self.output_size)
    outputs = []

    if self.verbose:
      print("Decoder State Size: ", decoder_state.shape)
      print("Decoder Input Size: ", decoder_input.shape)
    
    for i in range(max_output_chars):

      out, decoder_state = self.decoder_rnn_cell(decoder_input,decoder_state)
      if self.verbose:
        print("Decoder Intermediate Output Size: ", out.shape)
      
      out = self.hidden_2_output(decoder_state)
      out = self.softmax(out)
      outputs.append(out.view(1,-1))

      if self.verbose:
        print('Decoder Output size: ',out.shape)
        self.verbose = False

      max_idx = torch.argmax(out, 2, keepdim = True)
      if not ground_truth is None:
        max_idx = ground_truth[i].reshape(1,1,1)
      one_hot = torch.FloatTensor(out.shape).to(device)
      one_hot.zero_()
      one_hot.scatter_(2,max_idx,1)

      decoder_input = one_hot.detach()

    return outputs    

#Encoder Decoder using Attention

In [ ]:
class Transliteration_EncoderDecoder_Attention(nn.Module):
  def __init__(self,input_size, hidden_size, output_size,verbose = False):
    super(Transliteration_EncoderDecoder_Attention,self).__init__()

    self.hidden_size = hidden_size
    self.output_size = output_size

    self.encoder_rnn_cell = nn.GRU(input_size,hidden_size)
    self.decoder_rnn_cell = nn.GRU(hidden_size*2, hidden_size)

    self.hidden_2_output = nn.Liner(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=2)

    self.U = nn.Linear(self.hidden_size, self.hidden_size)
    self.W = nn.Linear(self.hidden_size, self.hidden_size)
    self.attention = nn.Linear(self.hidden_size, 1)
    self.output_2_hidden = nn.Linear(self.output_size, self.hidden_size)

    self.verbose = verbose

  def forward(self, input,max_out_chars = MAX_OUT_CHARS, device = 'cpu', ground_truth = None):
    # Encoder
    encoder_outputs, hidden = self.encoder_rnn_cell(input)
    encoder_outputs = encoder_outputs.view(-1, self.hidden_size)

    if self.verbose:
      print("Encoder Output Size: ", encoder_outputs.shape)

    #Decoder
    decoder_state = hidden
    decoder_input = torch.zeros(1, 1, self.output_size).to(device)

    outputs = []
    U = self.U(encoder_outputs)

    if self.verbose:
      print("Decoder State size: ", decoder_state.shape)
      print("Decoder Intermediate Input size: ", decoder_input.shape)
      print("U * Encoder Output size: ", U.shape)

    for i in range(max_output_chars):
      W = self.W(decoder_state.view(1, -1).repeat(encoder_outputs.shape[0], 1))
      V = self.attention(torch.tanh(U + W))
      attn_weights = F.softmax(V.view(1, -1), dim = 1) 
      
      if self.verbose:
          print("W * Decoder state Size", W.shape)
          print("V size", V.shape)
          print("Attention size", attn_weights.shape)
      
      attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                            encoder_outputs.unsqueeze(0))
      
      embedding = self.output_2_hidden(decoder_input)
      decoder_input = torch.cat((embedding[0], attn_applied[0]), 1).unsqueeze(0)

      if self.verbose:
          print('Attn LC', attn_applied.shape)
          print('Decoder input', decoder_input.shape)

      out, decoder_state = self.decoder_rnn_cell(decoder_input, decoder_state)

      if self.verbose:
          print('Decoder intermediate output', out.shape)

      out = self.h2o(decoder_state)
      out = self.softmax(out)
      outputs.append(out.view(1, -1))

      if self.verbose:
        print('Decoder output', out.shape)
        self.verbose = False

      max_idx = torch.argmax(out, 2, keepdim=True)
      if not ground_truth is None:
          max_idx = ground_truth[i].reshape(1, 1, 1)
      one_hot = torch.zeros(out.shape, device=device)
      one_hot.scatter_(2, max_idx, 1) 

      decoder_input = one_hot.detach()

  return outputs

      

In [ ]:
net = Transliteration_EncoderDecoder(len(eng_char2index), 256 ,len(hindi_char2index), verbose=True)

In [ ]:
print(net)

Transliteration_EncoderDecoder(
  (encoder_rnn_cell): GRU(27, 256)
  (decoder_rnn_cell): GRU(129, 256)
  (hidden_2_output): Linear(in_features=256, out_features=129, bias=True)
  (softmax): LogSoftmax(dim=2)
)


In [ ]:
def infer(net, word, char_limit, device = 'cpu'):
    input = Encode_.encode_now(word)
    return net(input,char_limit)


out = infer(net,'INDIA',30)

Encoder Input Size:  torch.Size([6, 1, 27])
Encoder Hidden Size:  torch.Size([1, 1, 256])
Encoder Output Size:  torch.Size([6, 1, 256])
Decoder State Size:  torch.Size([1, 1, 256])
Decoder Input Size:  torch.Size([1, 1, 129])
Decoder Intermediate Output Size:  torch.Size([1, 1, 256])
Decoder Output size:  torch.Size([1, 1, 129])


Transliteration_EncoderDecoder(
  (encoder_rnn_cell): GRU(27, 129)
  (decoder_rnn_cell): GRU(129, 256)
  (hidden_2_output): Linear(in_features=256, out_features=129, bias=True)
  (softmax): LogSoftmax(dim=2)
)

In [ ]:
def train_batch(net, opt, criterion, batch_size, device = 'cpu', teacher_force = False):
  net.train().to(device)
  opt.zero_grad()
  eng_batch, hindi_batch = train_data.get_batch(batch_size)

  total_loss = 0 
  for i in range(batch_size):
    input = Encode_.encode_now(eng_batch[i])
    ground_truth = Encode_.encode_now(hindi_batch[i])

    outputs = net(input,ground_truth.shape[0],device, ground_truth= ground_truth if teacher_force else none)

    for index, output in enumerate(outputs):
      loss = criterion(output, ground_truth[index])/batch_size
      loss.backward(retain_graph = True)
      total_loss += loss

  opt.step()
  return total_loss/batch_size
  

In [ ]:
def train_setup(net, lr = 0.01, num_batches = 100 , batch_size = 10 , momentum = 0.9, display_freq = 5, device = 'cpu'):
  net = net.to(device)
  criterion = nn.NLLLoss(ignore_index = -1)
  opt = optim.Adam(net.parameters(), lr = lr)
  teacher_force_upto = num_batches//3

  loss_arr = np.zeros(num_batches + 1)

  for i in range(num_batches):
    loss_arr[i+1] = (loss_arr[i]*i + train_batch(net,opt,criterion,batch_size,device,teacher_force= i<teacher_force_upto)/(i+1)

    if i%display_freq == display_freq-1:
      clear_output(wait = True)

      print("Iteration num: ",i,"Loss val: ", loss_arr[i])
      plt.figure()
      plt.plot(loss_arr[1:i], '-*')
      plt.xlabel("Iteration")
      plt.ylabel("Loss")
      plt.show()
      print('\n\n')

  torch.save(net, 'Model.pt')
  return loss_arr

In [ ]:
def test(net,word,device = 'cpu'):
  net = net.eval().to(device)
  outputs = infer(net,word,30,device)
  hindi_output = ''

  for out in outputs:
    val, indices = out.topk(1)
    index = indices.tplist()[0][0]
    if index == 0:
      break
    hindi_char = hindi_alphabates[index+1]
    hindi_output += hindi_char

  print(word + ' - ' + hindi_output)

In [ ]:
def calc_accuracy(net, device = 'cpu'):
    net = net.eval().to(device)
    predictions = []
    accuracy = 0
    for i in range(len(test_data)):
        eng, hindi = test_data[i]
        gt = gt_rep(hindi, hindi_alpha2index, device)
        outputs = infer(net, eng, gt.shape[0], device)
        correct = 0
        for index, out in enumerate(outputs):
            val, indices = out.topk(1)
            hindi_pos = indices.tolist()[0]
            if hindi_pos[0] == gt[index][0]:
                correct += 1
        
        accuracy += correct/gt.shape[0]
    accuracy /= len(test_data)
    return accuracy